## Deuflhard Function

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [2]:
# Constants and Parameters
const n, a, b = 2, 11, 10
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Deuflhard # Objective function

Deuflhard (generic function with 1 method)

In [3]:
d = 8 # Initial Degree 
SMPL = 40 # Number of samples
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), scale_factor, 1.0, f)
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

current L2-norm: 0.009384317330376233
Number of samples: 40
current L2-norm: 0.009679838314339191
Number of samples: 40


In [4]:
@polyvar(x[1:n]); # Define polynomial ring 

In [5]:
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Tracking 49 paths...   4%|█▎                            |  ETA: 0:04:39



Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:12
  # paths tracked:                  49
  # non-singular solutions (real):  49 (9)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (9)
Tracking 49 paths...   4%|█▎                            |  ETA: 0:01:54



Tracking 49 paths...  12%|███▋                          |  ETA: 0:00:39



Tracking 49 paths...  33%|█████████▊                    |  ETA: 0:00:12



Tracking 49 paths...  47%|██████████████▏               |  ETA: 0:00:07



Tracking 49 paths...  69%|████████████████████▉         |  ETA: 0:00:03



Tracking 49 paths...  86%|█████████████████████████▊    |  ETA: 0:00:01



Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:06
  # paths tracked:                  49
  # non-singular solutions (real):  49 (13)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 

Row,x1,x2,z
,Float64,Float64,Float64
1,-0.375961,0.98458,0.130242
2,-0.772423,0.772423,0.0887214
3,-0.655946,0.865381,0.206448
4,0.98458,-0.375961,0.130242
5,0.394318,0.394318,2.68186
6,-0.98458,0.375961,0.130242
7,-0.394318,-0.394318,2.68186
8,0.772423,-0.772423,0.0887214
9,0.655946,-0.865381,0.206448


In [8]:
process_critical_points(real_pts_cheb, f, scale_factor)

Row,x1,x2,z
,Float64,Float64,Float64
1,0.39892,0.39892,2.65526
2,-0.780329,0.780329,0.144272
3,-0.39892,-0.39892,2.65526
4,-0.998837,0.316356,0.0424662
5,-0.316356,0.998837,0.0424662
6,0.316356,-0.998837,0.0424662
7,0.998837,-0.316356,0.0424662
8,0.780329,-0.780329,0.144272
9,1.04917e-15,-4.33605e-15,4.0


In [13]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Deuflhard Function",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x1,
        scale_factor .* df_cheb.x2,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, scale_factor .* df_lege.x1,
        scale_factor .* df_lege.x2,
        df_lege.z,
        markersize=10,
        color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

2024-12-10 17:05:14.963 julia[84023:2180828] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-10 17:05:16.722 julia[84023:2180828] +[IMKInputSession subclass]: chose IMKInputSession_Modern


GLMakie.Screen(...)

In [10]:
# GLMakie.closeall()